In [1]:
root = r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle'

In [2]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
import json

In [3]:
class BeagleDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir , transforms = None):
        self.data_dir = data_dir
        self.transforms = transforms

        #load the annotations file, it also contain information of image names
        # load the annotations
        annotations1 = json.load(open(os.path.join(data_dir, 'via_region_data.json')))
        self.annotations = list(annotations1.values())

    def __getitem__(self,idx):
        #get the image path from the annotations data
        img_name = self.annotations[idx]['filename']
        img_path = os.path.join(self.data_dir,img_name)
        img = Image.open(img_path).convert("RGB")

        #first id is the background , objects count from 1
        obj_ids = np.array(range(len(self.annotations[idx]['regions']))) + 1

        #get bounding box coordinates for each objects
        num_objs = len(obj_ids)
        boxes = []

        for i in range(num_objs):
            xmin = np.min(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_x"])
            xmax = np.max(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_x"])
            ymin = np.min(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_y"])
            ymax = np.max(self.annotations[idx]["regions"][i]["shape_attributes"]["all_points_y"])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype = torch.float32)

        labels = torch.ones((num_objs,),dtype = torch.int64)

        image_id = torch.tensor([idx])
        area = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])

        iscrowd = torch.zeros((num_objs, ), dtype = torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = image_id
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.annotations)

In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def build_model(num_classses):
    #load the instance segmentation

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

    #get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
    

In [ ]:
dataset = BeagleDataset(r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle\train', get_transform(train= True))
dataset_test = BeagleDataset(r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle\val', get_transform(train= False))

In [13]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

# use our dataset and defined transformations
dataset = BeagleDataset(r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle\train', get_transform(train=True))
dataset_test = BeagleDataset(r'C:\Users\akilarasan.p\Downloads\fine-tune-MaskRcnn-master\beagle\val', get_transform(train=False))

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=0,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=utils.collate_fn)

In [14]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [15]:

# our dataset has two classes only - background and beagle
num_classes = 2

# get the model using our helper function
model = build_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

In [17]:
# number of epochs
num_epochs = 10
 
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

Epoch: [0]  [ 0/38]  eta: 0:15:46  lr: 0.000140  loss: 0.1158 (0.1158)  loss_classifier: 0.0314 (0.0314)  loss_box_reg: 0.0621 (0.0621)  loss_objectness: 0.0004 (0.0004)  loss_rpn_box_reg: 0.0219 (0.0219)  time: 24.9039  data: 0.1146  max mem: 0
Epoch: [0]  [10/38]  eta: 0:11:58  lr: 0.001490  loss: 0.1240 (0.1323)  loss_classifier: 0.0243 (0.0257)  loss_box_reg: 0.0884 (0.0882)  loss_objectness: 0.0034 (0.0043)  loss_rpn_box_reg: 0.0118 (0.0140)  time: 25.6521  data: 0.0712  max mem: 0
Epoch: [0]  [20/38]  eta: 0:07:53  lr: 0.002840  loss: 0.1288 (0.1332)  loss_classifier: 0.0243 (0.0269)  loss_box_reg: 0.0897 (0.0916)  loss_objectness: 0.0027 (0.0047)  loss_rpn_box_reg: 0.0054 (0.0100)  time: 26.4038  data: 0.0631  max mem: 0
Epoch: [0]  [30/38]  eta: 0:03:31  lr: 0.004190  loss: 0.1114 (0.1249)  loss_classifier: 0.0222 (0.0259)  loss_box_reg: 0.0744 (0.0834)  loss_objectness: 0.0011 (0.0041)  loss_rpn_box_reg: 0.0083 (0.0115)  time: 26.9553  data: 0.0559  max mem: 0
Epoch: [0]  [37/

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# set to evaluation mode
torch.save(model, 'faster-rcnn-beagle.pt')
model.eval()
CLASS_NAMES = ['__background__', 'beagle']
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)